# 使用 Milvus 和 DeepSeek 构建 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

### 依赖与环境

In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

---

In [2]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

### 准备数据

我们使用 Milvus 文档 2.4.x 中的 FAQ 页面作为我们 RAG 中的私有知识库，这是一个简单 RAG 管道的良好数据源。

In [3]:
from glob import glob
import re

text_lines = []

def split_civil_code():
    # 使用正则表达式匹配标题和内容
    # 读取文件内容
    with open("mfd.md", "r", encoding="utf-8") as file:
        content = file.read()

    # 使用正则表达式按 `**第xxx条**` 分割
    sections = re.split(r'(\*\*第[零一二三四五六七八九十百千]+条\*\*)', content)

    # 将分割后的内容组合成列表
    result = []
    for i in range(1, len(sections), 2):
        result.append(sections[i] + sections[i+1].strip())

    return result

text_lines=split_civil_code()

FileNotFoundError: [Errno 2] No such file or directory: 'mfd.md'

In [ ]:
len(text_lines)

In [ ]:
text_lines = all_lines[0:20]

In [ ]:
text_lines

### 准备 LLM 和 Embedding 模型

DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。

In [ ]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

定义一个 embedding 模型，使用 `milvus_model` 来生成文本嵌入。我们以 `DefaultEmbeddingFunction` 模型为例，这是一个预训练的轻量级嵌入模型。

In [ ]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()


生成一个测试嵌入并打印其维度和前几个元素。

In [ ]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

In [ ]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

## 将数据加载到 Milvus

### 创建 Collection

In [ ]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_civil_code_demo.db")

collection_name = "rag_civil_code_collection"

关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。

检查 collection 是否已存在，如果存在则删除它。

In [ ]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

创建一个具有指定参数的新 collection。

如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

`metric_type` (距离度量类型):
     作用：定义如何计算向量之间的相似程度。
     例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
     选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。

 `consistency_level` (一致性级别):
     作用：定义数据写入后，读取操作能多快看到这些新数据。
     例如：
         `Strong` (强一致性): 总是读到最新数据，可能稍慢。
         `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
         `Session` (会话一致性): 自己写入的自己能立刻读到。
         `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
     选择依据：在数据实时性要求和系统性能之间做权衡。

简单来说：
 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [ ]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="COSINE",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [ ]:
###
#对以上条件做总结，和输出5个相关的标签， 用JSON格式输出
#{ 'summary': <内容>, 
#   'tags': [ '<tag1>',...] 
#}
import json 

system_prompt_summary = """
对每个user input做总结，和输出5个相关的标签， 用JSON格式输出。
## 限制
1. 只需要输出生成的JSON。
2. 不要输出任何解释。
3. 不要输出任何其他内容。
4. 总结内容不要超过20字。
5. 标签不要超过5个字。

## 格式
{ 
'summary': <内容>, 
'tags': [ '<tag1>',...] 
}
"""

def get_civil_code_summary(text):
    response = deepseek_client.chat.completions.create(
            model="deepseek-chat",
            messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": text},            
        ],
        response_format={
            'type': 'json_object'
        }
    )
    jsonText = response.choices[0].message.content.replace("```json", "").replace("```", "")
    jsonText
    return json.loads(jsonText)

In [ ]:
text_lines_summary = []
text_lines_tags = []

for line in text_lines:
    summary = get_civil_code_summary(line)
    text_lines_summary.append(summary['summary'])
    text_lines_tags.append(summary['tags'])

In [ ]:
print(len(text_lines_summary))
print((text_lines_tags))

In [ ]:
len(text_lines)

In [ ]:
from tqdm import tqdm
import re 

pattern = r'\*\*第[零一二三四五六七八九十百千]+条\*\*\s*'

data = []

clean_text_lines = [re.sub(pattern, '', line) for line in text_lines]

doc_embeddings = embedding_model.encode_documents(clean_text_lines)

for i, line in enumerate(tqdm(clean_text_lines, desc="Creating embeddings")):
    data.append({"id": i, 
                 "vector": doc_embeddings[i], 
                 "text": line, 
                 "summary": text_lines_summary[i], 
                 "tags": ",".join(text_lines_tags[i])})

milvus_client.insert(collection_name=collection_name, data=data)

## 构建 RAG

### 检索查询数据

我们指定一个关于 Milvus 的常见问题。

In [ ]:
question = "​​什么是“预告登记”？"

在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [ ]:
def search(question):
    query = [question] if isinstance(question, str) else question
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=embedding_model.encode_queries(
        query
    ),  # 将问题转换为嵌入向量
    limit=10,  # 返回前3个结果
    search_params={"metric_type": "COSINE", "params": {}},  # 内积距离
        output_fields=["text","tags","summary"],  # 返回 text 字段
    )
    return search_res   


In [ ]:
question = "不动产"
search_res = search(question)

In [ ]:
# 查询所有匹配的实体
fuzzy_query = 'text like "%{q}%"'
# fuzzy_query = "summary like '%预告登记%'"
print(fuzzy_query)
query_result = milvus_client.query(
    collection_name=collection_name,
    filter=fuzzy_query,
    output_fields=["text","tags","summary"],
    search_params={"metric_type": "COSINE", "params": {"search_list": 100}},
    limit=5,
    filter_params = {"q": "不动产"}
)

In [ ]:
query_result

In [ ]:
for res in search_res[0]:
    print(res)

让我们看一下查询的搜索结果

In [ ]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [ ]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [ ]:
context

In [ ]:
question

为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [ ]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [ ]:
USER_PROMPT

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [ ]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(f"Question: {question}")
print(f"Answer: \n{response.choices[0].message.content}")

In [ ]:
import jieba.posseg as pseg

# 待分词的句子
sentence = "什么是预告登记？"

# 分词
words = pseg.lcut(sentence)
print(words)
nouns = [word for word, flag in words if flag.startswith('n') or flag.startswith('v')]

# 输出结果
print(nouns)